# IBM Capstone Project - The Battle of Neighborhoods
## Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


## Download and Explore Dataset

Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

This dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572



In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

### Tranform the data into a pandas dataframe

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods.

In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues

In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20181020'

In [10]:
#https://developer.foursquare.com/docs/resources/categories
#Italian Restaurant = 4bf58dd8d48988d110941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_italian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d110941735')
newyork_venues_italian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Broadway Pizza & Pasta,40.878822,-73.904494,Pizza Place
2,Marble Hill,40.876551,-73.91066,Patricia's of Tremont,40.882543,-73.913501,Italian Restaurant
3,Marble Hill,40.876551,-73.91066,pinochio pizza,40.877868,-73.918948,Italian Restaurant
4,Marble Hill,40.876551,-73.91066,Nonno Tony's,40.885288,-73.909588,Italian Restaurant


In [11]:
newyork_venues_italian.shape

(1508, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_italian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_italian, 'red', map_newyork_italian)

map_newyork_italian

In [14]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [15]:
manhattan_grouped = newyork_venues_italian.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_italian['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,39,39,39,39,39,39
Carnegie Hill,50,50,50,50,50,50
Central Harlem,8,8,8,8,8,8
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,11,11,11,11,11,11
East Village,50,50,50,50,50,50


## 3. Analyze Each Neighborhood

In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_italian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_italian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant,Gourmet Shop,Italian Restaurant,Mediterranean Restaurant,New American Restaurant,Nightclub,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Swiss Restaurant,Wine Bar
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant,Gourmet Shop,Italian Restaurant,Mediterranean Restaurant,New American Restaurant,Nightclub,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Swiss Restaurant,Wine Bar
0,Battery Park City,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.051282,0.000000,0.025641,0.717949,0.00,0.00,0.00,0.179487,0.00,0.00,0.025641,0.000000,0.00,0.00,0.000000
1,Carnegie Hill,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.780000,0.00,0.00,0.00,0.160000,0.00,0.02,0.000000,0.000000,0.00,0.00,0.040000
2,Central Harlem,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.875000,0.00,0.00,0.00,0.125000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000
3,Chelsea,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.840000,0.00,0.00,0.00,0.100000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.040000
4,Chinatown,0.00,0.000000,0.02,0.060000,0.00,0.00,0.000000,0.000000,0.020000,0.040000,0.760000,0.00,0.00,0.00,0.100000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000
5,Civic Center,0.00,0.000000,0.02,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.060000,0.820000,0.00,0.00,0.00,0.060000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.020000
6,Clinton,0.00,0.000000,0.00,0.000000,0.00,0.02,0.000000,0.000000,0.000000,0.000000,0.860000,0.00,0.00,0.00,0.080000,0.00,0.00,0.000000,0.020000,0.00,0.00,0.020000
7,East Harlem,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.636364,0.00,0.00,0.00,0.363636,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000
8,East Village,0.02,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.680000,0.00,0.00,0.00,0.200000,0.00,0.00,0.020000,0.000000,0.00,0.02,0.040000
9,Financial District,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.040000,0.000000,0.020000,0.720000,0.00,0.00,0.00,0.200000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.000000


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Italian Restaurant,Pizza Place,Event Space,Gourmet Shop,Sandwich Place,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
1,Carnegie Hill,Italian Restaurant,Pizza Place,Wine Bar,Restaurant,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar
2,Central Harlem,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
3,Chelsea,Italian Restaurant,Pizza Place,Wine Bar,Sandwich Place,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar
4,Chinatown,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Bakery,Fast Food Restaurant,Asian Restaurant,Cocktail Bar,Coffee Shop,Deli / Bodega


### Cluster Neighborhoods

In [20]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 4, 4, 1, 4, 4, 0, 0, 0])

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
1,Manhattan,Chinatown,40.715618,-73.994279,1,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Bakery,Fast Food Restaurant,Asian Restaurant,Cocktail Bar,Coffee Shop,Deli / Bodega
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Italian Restaurant,Swiss Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Italian Restaurant,Cocktail Bar,Swiss Restaurant,Asian Restaurant,Bakery,Café,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [23]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
7,East Harlem,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
11,Roosevelt Island,Italian Restaurant,Pizza Place,Sandwich Place,Wine Bar,Seafood Restaurant,Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar
19,East Village,Italian Restaurant,Pizza Place,Wine Bar,American Restaurant,Café,Swiss Restaurant,Sandwich Place,Event Space,Asian Restaurant,Bakery
27,Gramercy,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Mediterranean Restaurant,New American Restaurant,Wine Bar,Pub,Sandwich Place,Nightclub
28,Battery Park City,Italian Restaurant,Pizza Place,Event Space,Gourmet Shop,Sandwich Place,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
29,Financial District,Italian Restaurant,Pizza Place,Event Space,Gourmet Shop,Sandwich Place,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
37,Stuyvesant Town,Italian Restaurant,Pizza Place,Asian Restaurant,Sandwich Place,Café,Gourmet Shop,Bakery,Cocktail Bar,Coffee Shop,Deli / Bodega


### Cluster 1

In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Bakery,Fast Food Restaurant,Asian Restaurant,Cocktail Bar,Coffee Shop,Deli / Bodega
9,Yorkville,Italian Restaurant,Pizza Place,Wine Bar,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
10,Lenox Hill,Italian Restaurant,Pizza Place,Café,Cocktail Bar,Mediterranean Restaurant,Wine Bar,American Restaurant,Pub,Restaurant,Nightclub
16,Murray Hill,Italian Restaurant,Pizza Place,American Restaurant,Gourmet Shop,Steakhouse,Mediterranean Restaurant,Deli / Bodega,Event Space,Asian Restaurant,Bakery
20,Lower East Side,Italian Restaurant,Pizza Place,Fast Food Restaurant,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
30,Carnegie Hill,Italian Restaurant,Pizza Place,Wine Bar,Restaurant,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar
31,Noho,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Wine Bar,Swiss Restaurant,Restaurant,Event Space,Asian Restaurant,Bakery
33,Midtown South,Italian Restaurant,Pizza Place,Sandwich Place,Coffee Shop,New American Restaurant,Mediterranean Restaurant,Gourmet Shop,Event Space,Asian Restaurant,Bakery
34,Sutton Place,Italian Restaurant,Pizza Place,Wine Bar,Café,Nightclub,Cocktail Bar,Mediterranean Restaurant,Sandwich Place,Seafood Restaurant,Event Space
38,Flatiron,Italian Restaurant,Pizza Place,Sandwich Place,Pub,New American Restaurant,Mediterranean Restaurant,Gourmet Shop,Event Space,Asian Restaurant,Bakery


### Cluster 2

In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Italian Restaurant,Swiss Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant
5,Manhattanville,Italian Restaurant,Swiss Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant
18,Greenwich Village,Italian Restaurant,Sandwich Place,Gourmet Shop,Pizza Place,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
21,Tribeca,Italian Restaurant,Sandwich Place,Mediterranean Restaurant,Pizza Place,Wine Bar,Restaurant,Event Space,Asian Restaurant,Bakery,Café
26,Morningside Heights,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space


### Cluster 3

In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Italian Restaurant,Cocktail Bar,Swiss Restaurant,Asian Restaurant,Bakery,Café,Coffee Shop,Deli / Bodega,Event Space,Fast Food Restaurant


### Cluster 4

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
6,Central Harlem,Italian Restaurant,Pizza Place,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Event Space
8,Upper East Side,Italian Restaurant,Pizza Place,Café,American Restaurant,Sandwich Place,Restaurant,Asian Restaurant,Bakery,Steakhouse,Cocktail Bar
12,Upper West Side,Italian Restaurant,Pizza Place,Wine Bar,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
13,Lincoln Square,Italian Restaurant,Pizza Place,Seafood Restaurant,Deli / Bodega,Gourmet Shop,Asian Restaurant,Bakery,Café,Cocktail Bar,Coffee Shop
14,Clinton,Italian Restaurant,Pizza Place,Coffee Shop,Wine Bar,Seafood Restaurant,Sandwich Place,Event Space,Asian Restaurant,Bakery,Café
15,Midtown,Italian Restaurant,Pizza Place,Coffee Shop,Deli / Bodega,Gourmet Shop,American Restaurant,Steakhouse,Pub,Restaurant,Nightclub
17,Chelsea,Italian Restaurant,Pizza Place,Wine Bar,Sandwich Place,Seafood Restaurant,Fast Food Restaurant,Asian Restaurant,Bakery,Café,Cocktail Bar
22,Little Italy,Italian Restaurant,Pizza Place,Café,Gourmet Shop,Wine Bar,Sandwich Place,Restaurant,Event Space,Asian Restaurant,Bakery
23,Soho,Italian Restaurant,Pizza Place,Gourmet Shop,Café,Wine Bar,Sandwich Place,Restaurant,Event Space,Asian Restaurant,Bakery
